In [ ]:
# https://github.com/tuongv-1736461/EE399

In [22]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Generate data points
X = np.arange(0, 31)
Y = np.array([30, 35, 33, 32, 34, 37, 39, 38, 36, 36, 37, 39, 42, 45, 45, 41, 40, 39, 42, 44, 47, 49, 50, 49, 46, 48, 50, 53, 55, 54, 53])

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, 10)
        self.fc2 = nn.Linear(10, 10)
        self.fc3 = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Split the data into training and test sets
X_train_1, Y_train_1 = X[:20], Y[:20]
X_test_1, Y_test_1 = X[10:], Y[10:]

X_train_2 = np.concatenate((X[:10], X[-10:]))
Y_train_2 = np.concatenate((Y[:10], Y[-10:]))
X_test_2 = X[10:-10]
Y_test_2 = Y[10:-10]

# Define function to train and evaluate a model 
def train_and_evaluate(X_train, Y_train, X_test, Y_test):
  # Convert the training and test data to tensors
  X_train_tensor = torch.Tensor(X_train).unsqueeze(1)
  Y_train_tensor = torch.Tensor(Y_train).unsqueeze(1)
  X_test_tensor = torch.Tensor(X_test).unsqueeze(1)
  Y_test_tensor = torch.Tensor(Y_test).unsqueeze(1)

  # Create a new neural network
  net = Net()

  # Define the loss function
  criterion = nn.MSELoss()

  # Define the optimizer
  optimizer = optim.SGD(net.parameters(), lr=0.01)

  # Set the number of epochs
  num_epochs = 10

  # Training loop
  for epoch in range(num_epochs):
      optimizer.zero_grad()
      outputs = net(X_train_tensor)
      loss = criterion(outputs, Y_train_tensor)
      loss.backward()
      optimizer.step()

      # Print the loss at every 100th epoch
      if (epoch+1) % 100 == 0:
          print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

  # Evaluate the model on training and test data
  with torch.no_grad():
      train_predictions = net(X_train_tensor)
      train_loss = criterion(train_predictions, Y_train_tensor)
      test_predictions = net(X_test_tensor)
      test_loss = criterion(test_predictions, Y_test_tensor)

  # Print the training and test errors
  print(f'Train Error: {train_loss.item()}')
  print(f'Test Error: {test_loss.item()}')

# Call the function to train and evaluate the model
train_and_evaluate(X_train_1, Y_train_1, X_test_1, Y_test_1)
train_and_evaluate(X_train_2, Y_train_2, X_test_2, Y_test_2)





Train Error: 16.6630859375
Test Error: 91.0152359008789
Train Error: 142.67678833007812
Test Error: 66.91342163085938


In [20]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from sklearn.decomposition import PCA

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(20, 128)   # Adjusting input size to 20
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = torch.relu(self.fc1(x.float()))  # Convert input to float
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Load the MNIST dataset and apply transformations
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

# Get the data tensors from the datasets
X_train = train_dataset.data.float()  # Convert image data to float
Y_train = train_dataset.targets

# Reshape the input images for PCA (flatten each image)
X_train_flattened = X_train.view(X_train.size(0), -1)

# Compute PCA on the flattened images
pca = PCA(n_components=20)  # Number of PCA modes to compute (20 in this case)
X_train_pca = pca.fit_transform(X_train_flattened)

# Create data loaders for training and testing
train_data = torch.utils.data.TensorDataset(torch.from_numpy(X_train_pca), Y_train)
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=64, shuffle=True)

# Initialize the network and define the loss function and optimizer
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

# Train the network
num_epochs = 10
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(train_loader), loss.item()))

# Test the network
test_data = test_dataset.data.float()  # Convert image data to float
X_test_flattened = test_data.view(test_data.size(0), -1)
X_test_pca = pca.transform(X_test_flattened)  # Apply PCA transformation on test data
Y_test = test_dataset.targets

test_data = torch.utils.data.TensorDataset(torch.from_numpy(X_test_pca), Y_test)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=64, shuffle=False)

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.float()  # Convert images to float
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Epoch [1/10], Step [100/938], Loss: 1.1059
Epoch [1/10], Step [200/938], Loss: 1.4078
Epoch [1/10], Step [300/938], Loss: 1.0622
Epoch [1/10], Step [400/938], Loss: 0.9033
Epoch [1/10], Step [500/938], Loss: 0.6788
Epoch [1/10], Step [600/938], Loss: 0.5067
Epoch [1/10], Step [700/938], Loss: 0.3921
Epoch [1/10], Step [800/938], Loss: 0.4774
Epoch [1/10], Step [900/938], Loss: 0.3019
Epoch [2/10], Step [100/938], Loss: 0.4841
Epoch [2/10], Step [200/938], Loss: 0.4570
Epoch [2/10], Step [300/938], Loss: 0.2417
Epoch [2/10], Step [400/938], Loss: 0.3802
Epoch [2/10], Step [500/938], Loss: 0.2282
Epoch [2/10], Step [600/938], Loss: 0.2884
Epoch [2/10], Step [700/938], Loss: 0.2668
Epoch [2/10], Step [800/938], Loss: 0.3875
Epoch [2/10], Step [900/938], Loss: 0.2695
Epoch [3/10], Step [100/938], Loss: 0.1215
Epoch [3/10], Step [200/938], Loss: 0.2390
Epoch [3/10], Step [300/938], Loss: 0.2877
Epoch [3/10], Step [400/938], Loss: 0.2700
Epoch [3/10], Step [500/938], Loss: 0.2743
Epoch [3/10

In [21]:
from torch.utils.data import TensorDataset

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load MNIST dataset and apply transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

# Perform PCA on the dataset
train_images = train_dataset.data.reshape(-1, 28 * 28).numpy()
pca = PCA(n_components=20)
train_pca = pca.fit_transform(train_images)

test_images = test_dataset.data.reshape(-1, 28 * 28).numpy()
test_pca = pca.transform(test_images)

# Create new datasets with PCA-transformed data
train_pca_dataset = TensorDataset(torch.from_numpy(train_pca).float(), train_dataset.targets)
test_pca_dataset = TensorDataset(torch.from_numpy(test_pca).float(), test_dataset.targets)

# Model parameters
input_size = 20  # PCA component size
hidden_size = 128
num_layers = 2
num_classes = 10
batch_size = 64  # Define the batch size here
num_epochs = 5

# Define data loaders with PCA datasets
train_loader = torch.utils.data.DataLoader(dataset=train_pca_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_pca_dataset, batch_size=batch_size, shuffle=False)

# Define LSTM model architecture with modified input size
class LSTMNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTMNet, self).__init__()
        self.input_size = input_size  # Assign input_size to an attribute
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        batch_size = x.size(0)
        x = x.view(batch_size, -1, self.input_size)  # Reshape the input tensor
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

# Initialize the LSTM model with modified input size
model = LSTMNet(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on the test set: {accuracy:.2f}%')



Epoch [1/5], Step [100/938], Loss: 0.6845
Epoch [1/5], Step [200/938], Loss: 0.3719
Epoch [1/5], Step [300/938], Loss: 0.4686
Epoch [1/5], Step [400/938], Loss: 0.3301
Epoch [1/5], Step [500/938], Loss: 0.2429
Epoch [1/5], Step [600/938], Loss: 0.2855
Epoch [1/5], Step [700/938], Loss: 0.2752
Epoch [1/5], Step [800/938], Loss: 0.3036
Epoch [1/5], Step [900/938], Loss: 0.2735
Epoch [2/5], Step [100/938], Loss: 0.2294
Epoch [2/5], Step [200/938], Loss: 0.1860
Epoch [2/5], Step [300/938], Loss: 0.2787
Epoch [2/5], Step [400/938], Loss: 0.1761
Epoch [2/5], Step [500/938], Loss: 0.1880
Epoch [2/5], Step [600/938], Loss: 0.1296
Epoch [2/5], Step [700/938], Loss: 0.2077
Epoch [2/5], Step [800/938], Loss: 0.1525
Epoch [2/5], Step [900/938], Loss: 0.1116
Epoch [3/5], Step [100/938], Loss: 0.3791
Epoch [3/5], Step [200/938], Loss: 0.1767
Epoch [3/5], Step [300/938], Loss: 0.0718
Epoch [3/5], Step [400/938], Loss: 0.1674
Epoch [3/5], Step [500/938], Loss: 0.2880
Epoch [3/5], Step [600/938], Loss:

In [18]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from itertools import combinations
from sklearn.metrics import accuracy_score


# Load the dataset
mnist = fetch_openml('mnist_784')
X = mnist.data
y = mnist.target.astype(int)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


from sklearn.decomposition import PCA
# Apply PCA to reduce the dimensionality of the data
pca = PCA(n_components=20)
X_pca = pca.fit_transform(X)
# Project the data onto 20 modes PCA space
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Create a function to train and test classifiers on digit pairs
def test_digit_pairs(digit_pairs, classifier):
    accuracy_scores = []
    
    for digit_pair in digit_pairs:
        # Get the indices for the current digit pair
        digit_indices = ((y_train == digit_pair[0]) | (y_train == digit_pair[1]))

        # Create the feature and target datasets for the current digit pair
        X_pair_train = X_train_pca[digit_indices]
        y_pair_train = y_train[digit_indices]
        X_pair_test = X_test_pca[((y_test == digit_pair[0]) | (y_test == digit_pair[1]))]
        y_pair_test = y_test[((y_test == digit_pair[0]) | (y_test == digit_pair[1]))]

        # Train the classifier on the training data
        clf = classifier
        clf.fit(X_pair_train, y_pair_train)

        # Predict the labels of the test data
        y_pred = clf.predict(X_pair_test)

        # Calculate the accuracy of the classifier
        accuracy = accuracy_score(y_pair_test, y_pred)

        # Append the accuracy to the list of accuracy scores
        accuracy_scores.append(accuracy)

    # Calculate the average accuracy for all digit pairs
    avg_accuracy = sum(accuracy_scores) / len(accuracy_scores)
    print(f"Average accuracy for all digit pairs: {avg_accuracy:.5f}")

# Test the SVM classifier on all digit pairs using 20 modes PCA
svm_pairs = list(combinations(range(10), 2))
svm_classifier = SVC()
print("SVM")
test_digit_pairs(svm_pairs, svm_classifier)

# Test the decision tree classifier on all digit pairs
dt_pairs = list(combinations(range(10), 2))
dt_classifier = DecisionTreeClassifier(random_state=42)
print("Decision Tree")
test_digit_pairs(dt_pairs, dt_classifier)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


SVM
Average accuracy for all digit pairs: 0.99559
Decision Tree
Average accuracy for all digit pairs: 0.96874
